In [1]:
import pandas as pd
import numpy as np
import random as rn
import matplotlib.pyplot as plt
import seaborn as sns
import math, random, json
from pathlib import Path
from datetime import datetime
import statistics
import joblib

import sys
sys.path.append('./rtaUtils')

from rtaUtils import paths, experiment, data_loading

import tensorflow as tf
# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout
# from keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Reproducible
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(1234)
tf.random.set_seed(42)

In [3]:
detailed_metrics = True
num_prev_examples = 1
times = (15,30,60,90,120,150) # Minutes

# train_data_path = Path('../data/train/train_data_with_val')
# test_data_path  = Path('../data/test/test_data_with_val')
# val_data_path   = Path('../data/val/validation_data')
# models_path     = Path('./models')

# Feature selection
### Features ##################################################################
numeric_feat = [
    'latitude', 'longitude', 'altitude', 'departureDelay', 'vspeed', 'speed', 
    'day_of_week', 'track', 'wind_dir_degrees', 'wind_speed_kt', 
    'visibility_statute_mi', 'max_temp', 'min_temp', 'clouds', 'hav_distance'
]
categoric_feat = [
    'time_of_day', 'operator', 'aerodromeOfDeparture', 'sky_status'
]
objective = ['RTA']
num_features     = len(numeric_feat+categoric_feat)

encoders = joblib.load(paths.utils_path / 'encoder_19.joblib')

C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.1.3 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Model training

In [4]:
model = RandomForestRegressor(
    criterion = 'squared_error',
    n_estimators=50,
    n_jobs=6,
    verbose=1)

In [5]:
train_data = data_loading.load_final_data(month='20220[12345]', dataset='train', sampling = 30)
# encoders = {}

for feat in categoric_feat:
    le = LabelEncoder().fit(train_data[feat])
    # encoders[feat] = le
    
    train_data[feat] = le.transform(train_data[feat]).reshape(-1,1)
    
model.fit(
    train_data[numeric_feat+categoric_feat], 
    train_data[objective].values.reshape((-1,)))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  6.7min
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:  8.4min finished


RandomForestRegressor(n_estimators=50, n_jobs=6, verbose=1)

In [6]:
train_data = data_loading.load_final_data(month='20220[6789]', dataset='train', sampling = 30)
# encoders = {}

for feat in categoric_feat:
    le = LabelEncoder().fit(train_data[feat])
    # encoders[feat] = le
    
    train_data[feat] = le.transform(train_data[feat]).reshape(-1,1)
    
model.fit(
    train_data[numeric_feat+categoric_feat], 
    train_data[objective].values.reshape((-1,)))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  5.8min
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:  7.3min finished


RandomForestRegressor(n_estimators=50, n_jobs=6, verbose=1)

In [7]:
val_data   = data_loading.load_final_data(month='*', dataset='val', sampling = 30)
for feat in categoric_feat:
    le = encoders[feat]
    val_data[feat] = le.transform(val_data[feat]).reshape(-1,1)

mae = mean_absolute_error(
        val_data[objective], 
        model.predict(val_data[numeric_feat+categoric_feat]))
print('MAE val set:\t\t{:.5f} segundos'.format(mae))

rmse = math.sqrt(
    mean_squared_error(
        val_data[objective], 
        model.predict(val_data[numeric_feat+categoric_feat])))
print('RMSE val set:\t\t{:.5f} segundos'.format(rmse))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.3s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    4.0s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.


MAE val set:		256.68255 segundos


[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.3s


RMSE val set:		498.27843 segundos


[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    4.1s finished


In [8]:
data_at_times = {x:{'X':[], 'Y':[]} for x in times}
    
total_legs = len(val_data.fpId.unique())
for idx,fpId in enumerate(val_data.fpId.unique()):
    flight = val_data[val_data.fpId == fpId]
    if (idx+1)%25==0:
        print('{}/{}'.format(idx+1,total_legs).ljust(10,' '), end='\r')

    for t in times:
        f = flight[flight.RTA >= t*60]
        f = f.iloc[-num_prev_examples:]

        if len(f)<num_prev_examples: 
            continue
        
        for i in range(num_prev_examples):
            data_at_times[t]['X'].append(f.iloc[i][numeric_feat+categoric_feat])
            data_at_times[t]['Y'].append(f.iloc[i][objective])

print('{}/{} - Done'.format(idx+1,total_legs).ljust(10,' '), end='\r')
for t in times:
    for k,v in data_at_times[t].items():
        data_at_times[t][k] = np.array(data_at_times[t][k])

In [9]:
print('MAE')
for t in times:
    print('{}min: {:.3f}s'.format(t,mean_absolute_error(data_at_times[t]['Y'], 
                                                        model.predict(data_at_times[t]['X']))))

MAE
15min: 192.643s
30min: 214.200s
60min: 238.863s
90min: 323.193s
120min: 382.364s
150min: 399.939s


C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings

In [10]:
test_data   = data_loading.load_final_data(month='*', dataset='test', sampling = 30)
for feat in categoric_feat:
    le = encoders[feat]
    test_data[feat] = le.transform(test_data[feat]).reshape(-1,1)

mae = mean_absolute_error(
        test_data[objective], 
        model.predict(test_data[numeric_feat+categoric_feat]))
print('MAE test set:\t\t{:.5f} segundos'.format(mae))

rmse = math.sqrt(
    mean_squared_error(
        test_data[objective], 
        model.predict(test_data[numeric_feat+categoric_feat])))
print('RMSE test set:\t\t{:.5f} segundos'.format(rmse))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    4.2s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    5.1s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.


MAE test set:		256.92041 segundos


[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    4.3s


RMSE test set:		552.01595 segundos


[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    5.4s finished


In [11]:
test_data_at_times = {x:{'X':[], 'Y':[]} for x in times}
    
total_legs = len(test_data.fpId.unique())
for idx,fpId in enumerate(test_data.fpId.unique()):
    flight = test_data[test_data.fpId == fpId]
    if (idx+1)%25==0:
        print('{}/{}'.format(idx+1,total_legs).ljust(10,' '), end='\r')

    for t in times:
        f = flight[flight.RTA >= t*60]
        f = f.iloc[-num_prev_examples:]

        if len(f)<num_prev_examples: 
            continue
        
        for i in range(num_prev_examples):
            test_data_at_times[t]['X'].append(f.iloc[i][numeric_feat+categoric_feat])
            test_data_at_times[t]['Y'].append(f.iloc[i][objective])

print('{}/{} - Done'.format(idx+1,total_legs).ljust(10,' '), end='\r')
for t in times:
    for k,v in test_data_at_times[t].items():
        test_data_at_times[t][k] = np.array(test_data_at_times[t][k])

In [12]:
print('MAE')
for t in times:
    print('{}min: {:.3f}s'.format(t,mean_absolute_error(test_data_at_times[t]['Y'], 
                                                        model.predict(test_data_at_times[t]['X']))))
print('RMSE')
for t in times:
    print('{}min: {:.3f}s'.format(t,math.sqrt(mean_squared_error(test_data_at_times[t]['Y'], 
                                                        model.predict(test_data_at_times[t]['X'])))))

MAE
15min: 191.302s
30min: 225.646s
60min: 243.604s
90min: 320.148s
120min: 385.223s
150min: 395.168s
RMSE


C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings

15min: 693.321s
30min: 467.571s
60min: 543.278s
90min: 638.971s
120min: 534.188s
150min: 525.142s


[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklear

In [13]:
distances = (25,45,60,100,125,250)
test_data_at_distance = {x:{'X':[], 'Y':[]} for x in distances}
    
total_legs = len(test_data.fpId.unique())
for idx,fpId in enumerate(test_data.fpId.unique()):
    flight = test_data[test_data.fpId == fpId]
    if (idx+1)%25==0:
        print('{}/{}'.format(idx+1,total_legs).ljust(10,' '), end='\r')

    for t in distances:
        f = flight[flight.hav_distance >= t]
        f = f.iloc[-num_prev_examples:]

        if len(f)<num_prev_examples: 
            continue
        
        for i in range(num_prev_examples):
            test_data_at_distance[t]['X'].append(f.iloc[i][numeric_feat+categoric_feat])
            test_data_at_distance[t]['Y'].append(f.iloc[i][objective])
print('{}/{} - Done'.format(idx+1,total_legs).ljust(10,' '), end='\r')
for t in distances:
    for k,v in test_data_at_distance[t].items():
        test_data_at_distance[t][k] = np.array(test_data_at_distance[t][k])

In [14]:
print('MAE')
for t in distances:
    print('{}NM: {:.3f}s'.format(t,mean_absolute_error(test_data_at_distance[t]['Y'], 
                                                        model.predict(test_data_at_distance[t]['X']))))

print('rmse')
for t in distances:
    print('{}NM: {:.3f}s'.format(t,math.sqrt(mean_squared_error(test_data_at_distance[t]['Y'], 
                                                        model.predict(test_data_at_distance[t]['X'])))))

MAE
25NM: 127.342s
45NM: 173.510s
60NM: 215.690s
100NM: 248.065s
125NM: 248.702s
250NM: 218.168s
rmse


C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings

25NM: 568.444s
45NM: 780.658s
60NM: 765.669s
100NM: 544.768s
125NM: 515.271s
250NM: 431.925s


[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklear

In [15]:
# Future data
test_data   = data_loading.load_final_data(month='202210', dataset='test', sampling = 30)
for feat in categoric_feat:
    le = encoders[feat]
    test_data[feat] = le.transform(test_data[feat]).reshape(-1,1)

test_data_at_times = {x:{'X':[], 'Y':[]} for x in times}
    
total_legs = len(test_data.fpId.unique())
for idx,fpId in enumerate(test_data.fpId.unique()):
    flight = test_data[test_data.fpId == fpId]
    if (idx+1)%25==0:
        print('{}/{}'.format(idx+1,total_legs).ljust(10,' '), end='\r')

    for t in times:
        f = flight[flight.RTA >= t*60]
        f = f.iloc[-num_prev_examples:]

        if len(f)<num_prev_examples: 
            continue
        
        for i in range(num_prev_examples):
            test_data_at_times[t]['X'].append(f.iloc[i][numeric_feat+categoric_feat])
            test_data_at_times[t]['Y'].append(f.iloc[i][objective])

print('{}/{} - Done'.format(idx+1,total_legs).ljust(10,' '), end='\r')
for t in times:
    for k,v in test_data_at_times[t].items():
        test_data_at_times[t][k] = np.array(test_data_at_times[t][k])
        
mae = mean_absolute_error(
        test_data[objective], 
        model.predict(test_data[numeric_feat+categoric_feat]))
print('MAE test set:\t\t{:.5f} segundos'.format(mae))

rmse = math.sqrt(
    mean_squared_error(
        test_data[objective], 
        model.predict(test_data[numeric_feat+categoric_feat])))
print('RMSE test set:\t\t{:.5f} segundos'.format(rmse))
        
print('MAE')
for t in times:
    print('{}min: {:.3f}s'.format(t,mean_absolute_error(test_data_at_times[t]['Y'], 
                                                        model.predict(test_data_at_times[t]['X']))))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    3.1s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.


MAE test set:		313.48904 segundos


[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    2.5s


RMSE test set:		740.80278 segundos
MAE
15min: 272.066s
30min: 275.759s
60min: 248.115s
90min: 336.175s
120min: 473.826s
150min: 464.170s


[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    3.2s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature nam

In [ ]:


# Categorical to numerical conversion
encoders = {}
for feat in categoric_feat:
    le = LabelEncoder().fit(train_data[feat])
    encoders[feat] = le
    
    train_data[feat] = le.transform(train_data[feat]).reshape(-1,1)
    
# Normalization to [0,1] range
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_data[numeric_feat+categoric_feat+objective])

scaled_train_data = scaler.transform(train_data[numeric_feat+categoric_feat+objective])

# Conversion to supervised problem
# Data formatting
train_data['idx'] = train_data.index
# Assumes that data are sorted by flight/leg and timestamp.
indices_train = train_data.groupby('leg').agg(first=('idx', 'first'), last=('idx', 'last'))

train_X, train_Y = [],[]
for first, last in indices_train.values:
    temp_x, temp_y = create_dataset(scaled_train_data[first:last,:], lookback)
    train_X.append(temp_x)
    train_Y.append(temp_y)
    
train_X = np.concatenate(train_X, axis=0)
train_Y = np.concatenate(train_Y, axis=0)

In [ ]:

for feat in categoric_feat:
    val_data[feat]   = le.transform(val_data[feat]).reshape(-1,1)
scaled_val_data = scaler.transform(val_data[numeric_feat+categoric_feat+objective])

# Data formatting
val_data['idx'] = val_data.index
indices_val = val_data.groupby('leg').agg(first=('idx', 'first'), last=('idx', 'last'))

val_X, val_Y = [],[]
for first, last in indices_val.values:
    temp_x, temp_y = create_dataset(scaled_val_data[first:last,:], lookback)
    val_X.append(temp_x)
    val_Y.append(temp_y)

val_X = np.concatenate(val_X, axis=0)
val_Y = np.concatenate(val_Y, axis=0)

In [8]:
# Multiple messages
if detailed_metrics:
    data_at_times = {x:{'X':[], 'Y':[]} for x in times}
    
    total_legs = len(val_data.leg.unique())
    for idx,leg in enumerate(val_data.leg.unique()):
        flight = val_data[val_data.leg == leg]
        if (idx+1)%25==0:
            print('{}/{}'.format(idx+1,total_legs).ljust(10,' '), end='\r')

        for t in times:
            f = flight[flight.RTA >= t*60]
            f = f.iloc[-lookback-num_prev_examples:]
            
            if len(f)<lookback+num_prev_examples: 
                continue
            
            scaled_times_data = scaler.transform(f[numeric_feat+categoric_feat+objective])
            test_X, test_Y    = create_dataset(scaled_times_data, lookback)
            for i in range(num_prev_examples):
                data_at_times[t]['X'].append(test_X[i])
                data_at_times[t]['Y'].append(test_Y[i])
            
    print('{}/{} - Done'.format(idx+1,total_legs).ljust(10,' '), end='\r')
    for t in times:
        for k,v in data_at_times[t].items():
            data_at_times[t][k] = np.array(data_at_times[t][k])

In [47]:
for f,i in zip(val_data[numeric_feat+categoric_feat].columns, model.feature_importances_):
    print(f, '\t', i)


latitude 	 0.00029618901046454306
longitude 	 0.005216288208400189
altitude 	 0.0063576640106789055
departureDelay 	 0.0002514266013197549
vspeed 	 9.137399246544936e-05
speed 	 0.0035539660724611577
day_of_week 	 3.245619303266305e-05
track 	 0.00013495466836408365
wind_dir_degrees 	 0.0007883066696654517
wind_speed_kt 	 0.00033715693151600926
visibility_statute_mi 	 0.00011893221375816626
max_temp 	 9.370790532208846e-05
min_temp 	 0.00013742827328527973
clouds 	 1.529394837541134e-05
hav_distance 	 0.9816969655543435
time_of_day 	 8.717761596049648e-05
operator 	 0.0004263373690699054
aerodromeOfDeparture 	 0.00036417242674015597
sky_status 	 2.023347767616562e-07
